In [2]:
!python -m pip install dlib-19.24.1-cp311-cp311-win_amd64.whl

Processing c:\users\dimal\pycharmprojects\pythonproject\dlib-19.24.1-cp311-cp311-win_amd64.whl



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import dlib
import face_recognition as fr
from PIL import Image
import cv2
import numpy as np

In [9]:
class Photo:
    def __init__(self, img_path: str): #img_path - путь к файлу
        self.img = cv2.imread(img_path)
        self.pil_img = Image.open(img_path)
        self.size = self.pil_img.size

    def find_face(self, min_size=(30, 30), draw_rects=False):
        '''
        Метод нахождения лиц на фото.
        min_res - минимальное разрешение лица
        Возвращает класс rectangles.
        '''
        img_gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
        detector = dlib.get_frontal_face_detector()
        faces = detector(img_gray)
        Photo.check_size(faces, min_size)
        if draw_rects == True:
            Photo.draw_rects(self.img, faces)
        return faces

    @staticmethod
    def draw_rects(img, detection):
        '''
        Метод рисования прямоугольников вокруг лиц.
        img - изначальное фото
        detection - результат работы find_face()
        '''
        for face in detection:
            cv2.rectangle(img, (face.left(), face.top()), (face.right(), face.bottom()), (0, 0, 255), 2)

    def show_img(self, name: str, scale=None):
        new_img = self.img
        if scale is not None:
            new_img = cv2.resize(new_img, (int(self.size[0] * scale), int(self.size[1] * scale)))
        cv2.imshow(name, new_img)
        cv2.waitKey()

    @staticmethod
    def check_size(detection, size: tuple):
        filtered_faces = []
        for face in detection:
            if (face.right() - face.left()) >= size[0] or (face.bottom() - face.top()) >= size[1]:
                filtered_faces.append(face)
        return filtered_faces

    def photos(self, detection) -> tuple:
        return tuple([self.img[face.top():face.bottom(), face.left():face.right()] for face in detection])

        

In [18]:
class EncodingCreationError(Exception):
    def __init__(self, message="Failed to create encoding"):
        self.message = message
        super().__init__(self.message)


class Comparison:
    @staticmethod
    def faces_compare(image_array_1, image_array_2, jitters=30) -> float:
        # Загрузка изображений и получение кодировок лиц
        image_1 = np.array(image_array_1)
        image_2 = np.array(image_array_2)

        # Получение кодировок лиц с изображений
        encoding_1 = fr.face_encodings(image_1, num_jitters=jitters, model='large')
        encoding_2 = fr.face_encodings(image_2, num_jitters=jitters, model='large')

        if len(encoding_1) == 0 or len(encoding_2) == 0:
            # Если лица не найдены на одном из изображений
            raise EncodingCreationError

        # Берем первое найденное лицо на каждом изображении
        face_encoding_1 = encoding_1[0]
        face_encoding_2 = encoding_2[0]

        # Сравнение лиц
        face_distance = fr.face_distance([face_encoding_1], face_encoding_2)

        # Преобразуем расстояние в меру схожести от 0 до 1
        similarity = 1 - face_distance[0]  # Чем меньше расстояние, тем ближе к 1, т.е. 0 - наименьшая степень схожести, 1 - наибольшая степень схожести

        return similarity

In [19]:
photo = Photo('project_staff/faces_2.jpg')
faces = photo.photos(photo.find_face())
Comparison.faces_compare(faces[0], faces[1], 50)

0.3851158142090636

In [ ]:
0.38201283949330533